## Projeto de Redes Sociais

In [33]:
import freeman as fm
import pandas as pd
from unidecode import unidecode

In [23]:
athletes = pd.read_csv('data/crossfit/2019_opens_athletes.csv')

C:\Users\lchen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.



In [7]:
scores = pd.read_csv('data/crossfit/2019_opens_scores.csv')

In [26]:
athletes = athletes.dropna(subset=['affiliateid', 'affiliatename'])

gyms = {}
for _, row in athletes.iterrows():
    if (row['affiliateid'] in gyms.keys()):
        if (gyms[row['affiliateid']] != row['affiliatename']):
            print('Duplicate', gyms[row['affiliateid']], row['affiliatename'])
    gyms[row['affiliateid']] = row['affiliatename']

In [38]:
tmp = 'graph [\n  directed 0\n'

for key, value in gyms.items():
    tmp += '  node [\n    id ' + str(key) + '\n    affiliate_name "' + unidecode(value) + '"\n  ]\n'

tmp += ']'
filename = 'data/gyms.gml'
with open(filename, 'w') as f:
    f.write(tmp)

In [39]:
g = fm.load('data/gyms.gml')